In [1]:
import importlib
import movies
import data_describe
import os

importlib.reload(movies)

MoviesKG = movies.MoviesKG
DataDescribe = data_describe.DataDescribe


import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats("svg")

In [2]:
# # 创建实例并保存数据到JSON文件
# movies_kg = MoviesKG(download=True)
# movies_kg.save_to_json()

movies_kg = MoviesKG()
# movies_kg.clear_database()
movies_kg.filter_data()
# movies_kg.build_graph()
movies_kg.get_node_counts_by_label()

[{'label': 'Movie', 'node_count': 1967},
 {'label': 'Genre', 'node_count': 19},
 {'label': 'ProductionCompany', 'node_count': 2611},
 {'label': 'Country', 'node_count': 54},
 {'label': 'Person', 'node_count': 17540}]

In [22]:
movies_kg.filtered_data[0]

{'movie': {'belongs_to_collection': 'Inside Out Collection',
  'budget': 200000000,
  'id': 1022789,
  'overview': "Teenager Riley's mind headquarters is undergoing a sudden demolition to make room for something entirely unexpected: new Emotions! Joy, Sadness, Anger, Fear and Disgust, who’ve long been running a successful operation by all accounts, aren’t sure how to feel when Anxiety shows up. And it looks like she’s not alone.",
  'release_date': '2024-06-11',
  'revenue': 110800000,
  'runtime': 97,
  'tagline': 'Make room for new emotions.',
  'title': 'Inside Out 2'},
 'genres': ['Animation', 'Family', 'Drama', 'Adventure', 'Comedy'],
 'production_companies': [{'id': 3, 'name': 'Pixar', 'origin_country': 'US'},
  {'id': 2, 'name': 'Walt Disney Pictures', 'origin_country': 'US'}],
 'crew': [{'id': 182001, 'name': 'Kelsey Mann', 'job': 'Director'},
  {'id': 8103, 'name': 'Mark Nielsen', 'job': 'Producer'},
  {'id': 12890, 'name': 'Pete Docter', 'job': 'Executive Producer'},
  {'id':

In [11]:
from langchain_community.chat_models import ChatZhipuAI
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph

llm = ChatZhipuAI(
    temperature=0,
    zhipuai_api_key=os.getenv("ZHIPUAI_API_KEY"),
    model_name="GLM-4-0520",
    # model_name="GLM-4-Flash",
)
graph = Neo4jGraph(
    url="bolt://localhost:7687", username="neo4j", password=os.getenv("NEO4J_PASSWORD")
)
chain = GraphCypherQAChain.from_llm(llm, graph=graph, verbose=True)


def get_answer(question, choice=""):
    if choice:
        return chain.invoke(choice)["result"]
    return chain.invoke(question)["result"]

In [34]:
print(graph.get_schema)

Node properties:
Movie {budget: INTEGER, id: INTEGER, title: STRING, tagline: STRING, belongs_to_collection: STRING, runtime: INTEGER, overview: STRING, revenue: INTEGER, release_date: STRING}
Genre {name: STRING}
ProductionCompany {origin_country: STRING, name: STRING, id: INTEGER}
Country {name: STRING}
Person {name: STRING, id: INTEGER, gender: INTEGER}
Relationship properties:
ACTED_IN {character: STRING}
The relationships:
(:Movie)-[:BELONGS_TO_GENRE]->(:Genre)
(:Movie)-[:PRODUCED_BY]->(:ProductionCompany)
(:Movie)-[:ORIGIN_COUNTRY]->(:Country)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:EXECUTIVE_PRODUCER]->(:Movie)
(:Person)-[:DIRECTOR]->(:Movie)
(:Person)-[:PRODUCER]->(:Movie)


In [29]:
from enum import Enum


class QUESTION(str, Enum):
    # 简单单跳问题
    Q11 = "Who starred in 'The Dark Knight'?"
    Q12 = "Who directed 'The Dark Knight'?"
    Q13 = "Which movies did Christopher Nolan direct?"

    # 添加筛选条件
    Q21 = (
        "Besides 'The Dark Knight', which other movies has Christopher Nolan directed?"
    )
    Q22 = "In which movies have Michael Caine and Maggie Gyllenhaal co-starred?"
    # Q5 = "What other films has the director of 'The Dark Knight' directed?"


default_questions = [question.value for question in QUESTION]

print(get_answer(QUESTION.Q22.value))



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p1:Person {name: 'Michael Caine'})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(p2:Person {name: 'Maggie Gyllenhaal'})
RETURN m.title
Full Context:
[{'m.title': 'The Dark Knight'}]

> Finished chain.
Michael Caine and Maggie Gyllenhaal have co-starred in "The Dark Knight."


In [ ]:
import gradio as gr

# Corrected Gradio interface setup
iface = gr.Interface(
    fn=get_answer,
    inputs=[
        gr.Dropdown(
            choices=default_questions,
            label="Select a question",
            value=default_questions[0],
        ),  # 默认选择第一个问题
        gr.Textbox(lines=2, label="Or enter your question here..."),
    ],
    outputs=gr.Textbox(lines=10, label="Output"),  # 增加输出行数
    title="GraphCypher QA System",
    description="Ask any question related to the movie database.",
)


# Launch the Gradio app
iface.launch(share=True)

In [14]:
from langchain_community.chains.graph_qa.cypher import construct_schema

schema = construct_schema(graph.get_structured_schema, [], [])
print(schema)

Node properties are the following:
Movie {budget: INTEGER, id: INTEGER, title: STRING, tagline: STRING, belongs_to_collection: STRING, runtime: INTEGER, overview: STRING, revenue: INTEGER, release_date: STRING},Genre {name: STRING},ProductionCompany {origin_country: STRING, name: STRING, id: INTEGER},Country {name: STRING},Person {name: STRING, id: INTEGER, gender: INTEGER}
Relationship properties are the following:
ACTED_IN {character: STRING}
The relationships are the following:
(:Movie)-[:BELONGS_TO_GENRE]->(:Genre),(:Movie)-[:PRODUCED_BY]->(:ProductionCompany),(:Movie)-[:ORIGIN_COUNTRY]->(:Country),(:Person)-[:ACTED_IN]->(:Movie),(:Person)-[:EXECUTIVE_PRODUCER]->(:Movie),(:Person)-[:DIRECTOR]->(:Movie),(:Person)-[:PRODUCER]->(:Movie)
